# Kentro

In [1]:
# Import the CockpitATS client and required modules
import sys
import os
from datetime import datetime
import requests

# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.kentro import KentroClient

base_url = os.environ.get("KENTRO_BASE_URL")
username = os.environ.get("KENTRO_USERNAME")
password = os.environ.get("KENTRO_PASSWORD")
test_email = os.environ.get("KENTRO_TEST_EMAIL")

kentro_client = KentroClient()

Test get_candidate_id_from_email endpoint

In [2]:
candidate_id = kentro_client.get_candidate_id_from_email(test_email)
print(f"Candidate ID for {test_email}: {candidate_id}")

Candidate ID for TestFlexkrachtPvT00_Tim@pvt-impl.nl: 100000001


### Test endpoints with provided User IDs
- 100101678
- 100102195

In [7]:
candidate_id = os.environ.get("KENTRO_TEST_CANDIDATE_ID")

Payslips

In [8]:
payslip[0].amount

IndexError: list index out of range

In [9]:
payslip = kentro_client.get_payslips(candidate_id)
print(f"Found {len(payslip)} payslips for candidate ID {candidate_id}")
for item in payslip:
    print("\n")
    print(f"id: {item.id}") 
    print(f"frequency: {item.frequency}")
    print(f"amount: {item.amount}")

Found 25 payslips for candidate ID 100102195


id: 1223595
frequency: W
amount: 448.97


id: 1209941
frequency: W
amount: 404.99


id: 1194574
frequency: W
amount: 383.36


id: 1180618
frequency: W
amount: 223.92


id: 1167313
frequency: W
amount: 211.32


id: 1154330
frequency: W
amount: 561.07


id: 1139129
frequency: W
amount: 457.63


id: 1125205
frequency: W
amount: 453.81


id: 1112483
frequency: W
amount: 463.82


id: 1098585
frequency: W
amount: 515.44


id: 1083374
frequency: W
amount: 502.94


id: 1069947
frequency: W
amount: 511.45


id: 1056146
frequency: W
amount: 496.73


id: 1042630
frequency: W
amount: 432.18


id: 1027540
frequency: W
amount: 491.89


id: 1014629
frequency: W
amount: 516.26


id: 1001484
frequency: W
amount: 359.82


id: 988954
frequency: W
amount: 1031.55


id: 961865
frequency: W
amount: 499.02


id: 949331
frequency: W
amount: 471.69


id: 936890
frequency: W
amount: 488.36


id: 927260
frequency: W
amount: 62.84


id: 918252
frequency: W
amount: 48

Reservations

In [ ]:
reservations[0]

'Vakantieuren wettelijk lopend jaar'

In [11]:
reservations = kentro_client.get_reservation_balances(candidate_id)
for item in reservations:
    print("\n")
    print(f"Description: {item.description}")
    print(f"Year: {item.year}")
    print(f"Available Balance: {item.available_balance}")
    print(f"Unit: {item.withdrawal_unit}") # e.g., 'Hours' or 'EUR'
    print(f"Accrued Amount: {item.accrued_amount}")
    print(f"Type ID: {item.type_id}")



Description: Vakantieuren wettelijk lopend jaar
Year: 2025
Available Balance: None
Unit: None
Accrued Amount: 6.78
Type ID: 10064


Description: Vakantieuren bovenwettelijk
Year: 2025
Available Balance: 10.87
Unit: U
Accrued Amount: 10.87
Type ID: 10063


Description: Vakantiegeld
Year: 2025
Available Balance: 207.3
Unit: G
Accrued Amount: 207.3
Type ID: 10019


Description: Structurele toeslag vak.dagen
Year: 2025
Available Balance: 11.73
Unit: G
Accrued Amount: 11.73
Type ID: 10020


Description: Vakantieuren totaal
Year: 2025
Available Balance: 17.64
Unit: U
Accrued Amount: 17.65
Type ID: 10016


Contracts

In [16]:
contracts = kentro_client.get_contracts(candidate_id)
for item in contracts:
    print("\n")
    print(f"contract id: {item.id}") 
    print(f"start_date: {item.start_date}")
    print(f"end_date: {item.end_date}")
    print(f"status: {item.status}")
    print(f"function description: {item.function_description}")
    print(f"hourly_wage: {item.hourly_wage}")
    



contract id: 131594
start_date: 2025-02-10T00:00:00
end_date: 2025-03-09T00:00:00
status: Gesloten
function description: Warehouse Operative
hourly_wage: 14.7


contract id: 137146
start_date: 2025-03-10T00:00:00
end_date: 2025-06-15T00:00:00
status: Gesloten
function description: Warehouse Operative
hourly_wage: 14.7


contract id: 158187
start_date: 2025-06-16T00:00:00
end_date: 2025-12-14T00:00:00
status: Lopend
function description: Warehouse Operative
hourly_wage: 14.7


# Cockpit API

In [8]:
# Import the CockpitATS client and required modules
import sys
import os
from datetime import datetime

# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.cockpit import CockpitATSClient

print("Successfully imported CockpitATSClient")
print(f"Current time: {datetime.now()}")

cockpit_token = os.getenv("COCKPIT_KEY")
cockpit_client = CockpitATSClient("RN-f83d6706-edfb-4b90-a15d-f482ac23f716")

Successfully imported CockpitATSClient
Current time: 2025-11-14 14:52:00.662497


In [24]:
cockpit_client._get_bearer_token()

Error during Cockpit token exchange: 404 Client Error: Not Found for url: https://timing-prelive.recruitnowcockpit.nl/rest/api/v2/candidateprofile/api/authentication/token


HTTPError: 404 Client Error: Not Found for url: https://timing-prelive.recruitnowcockpit.nl/rest/api/v2/candidateprofile/api/authentication/token

In [ ]:
# Test CockpitATSClient (requires identity_provider_token)
print("Testing CockpitATSClient...")

# Example of how to test with a mock token (for demonstration purposes)
try:
    test_token = os.getenv("COCKPIT_KEY")
    cockpit_client = CockpitATSClient(test_token)
    
    # Test get_branch_details method
    print("\nTesting get_branch_details method...")
    try:
        branch_details = cockpit_client.get_branch_details()
        print("Successfully retrieved branch details:")
        print(f"   Response type: {type(branch_details)}")
        if isinstance(branch_details, dict):
            print(f"   Number of fields: {len(branch_details)}")
            print(f"   Fields: {list(branch_details.keys())}")
        else:
            print(f"   Response: {branch_details}")
    except Exception as branch_error:
        print(f"get_branch_details failed: {branch_error}")
        
except Exception as e:
    print(f"CockpitATSClient test failed: {e}")
    print("Make sure you have:")
    print("1. COCKPIT_BASE_URL environment variable set")
    print("2. A valid identity_provider_token (COCKPIT_KEY environment variable)")

Testing CockpitATSClient...

Testing get_branch_details method...
Error during Cockpit token exchange: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token
get_branch_details failed: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token


# Planbition

In [18]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import sys
import os
import requests

# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.planbition import PlanbitionClient

# Initialize the client (this will fail if environment variables are not set)
try:
    client = PlanbitionClient()
    print("Successfully initialized PlanbitionClient")
except ValueError as e:
    print(f"Failed to initialize PlanbitionClient: {e}")
    print("Make sure the following environment variables are set:")
    print("- PLANBITION_BASE_URL")
    print("- PLANBITION_KEY") 
    print("- PLANBITION_USERNAME")
    print("- PLANBITION_PASSWORD")
    client = None

Successfully initialized PlanbitionClient


Test endpoints

In [19]:
# Add this temporarily to your client to test
test_employee = client.get_employee_details("RE0036443")
test_employee

{'employee_id': 'RE0036443',
 'employee_number': 'RE0036443',
 'full_name': 'Malgorzata Zwawiak',
 'email': 'gosiak127@poczta.onet.pl',
 'country': 'Nederland',
 'is_active': True}

In [21]:
schedule = client.get_employee_schedule("RE0036443", "2025-05-01", "2025-05-30")
print(f"Found {len(schedule)} schedule items.")


Found 7 schedule items.


In [22]:
schedule

[ScheduleItem(id='DEF894FC-1A82-493D-AE67-7FD0C2E94177_627B699E-0FA6-4153-AC90-C32EBFC230DD_280520250730', employeeNumber='RE0036443', personId='RE0036443', organisationStructureId='Bleckmann Nederland - Nijverheidsstraat', organisationStructureName='Bleckmann Nederland - Nijverheidsstraat', customerId='0FDD57AF-5DA2-432A-B95F-C158CEBC822A', customerName='Bleckmann Nederland - Nijverheidsstraat', customerStructureName='Ruud', shiftId='F40D2C95-433D-4446-A200-E3A6D5E1A215', shiftName='Ruud Reco', shiftQualificationDemandId='627B699E-0FA6-4153-AC90-C32EBFC230DD', shiftQualificationDemandName='Ruud Reco', scheduleEmployeeAdministrativeComponents=[], startTime={'year': 2025, 'month': 5, 'day': 28, 'hour': 7, 'minute': 30, 'second': 0, 'millisecond': 0}, endTime={'year': 2025, 'month': 5, 'day': 28, 'hour': 16, 'minute': 30, 'second': 0, 'millisecond': 0}, plannerRemarkPublic=None, plannerRemarkPrivate=None, employeeRemark=None, iso_start_time='2025-05-28T07:30:00'),
 ScheduleItem(id='DEF89

In [8]:
# Print fields from the data model
for item in schedule:
    print("\n")
    print(f"Shift: {item.shiftName}") 
    print(f"Time: {item.iso_start_time}")
    print(f"ID: {item.customerName}")



Shift: Ruud Reco
Time: 2025-05-28T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-27T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-23T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-22T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-21T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-20T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-16T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat
